# Lesson 07 — A2A Client Fundamentals

Build a client that **discovers** and **communicates** with A2A agents.

## What You'll Learn

- Resolve an Agent Card with `A2ACardResolver`
- Send blocking requests with `A2AClient.send_message()`
- Stream responses with `A2AClient.send_message_streaming()`
- Parse Tasks, Messages, and Parts
- Handle errors gracefully

## Prerequisites

- **Lesson 06 server must be running** on `http://localhost:10001`
- Run: `cd ../06-a2a-server/src && python server.py`

> **A2A SDK docs:** [pypi.org/project/a2a-sdk](https://pypi.org/project/a2a-sdk/)


## Step 1 — Install Dependencies


In [ ]:
# ── Dependencies ──────────────────────────────────────────────────────────────
# a2a-sdk[http-server] → A2A Python SDK (includes A2AClient, A2ACardResolver)
# httpx                → Async HTTP client used directly in this lesson
# python-dotenv        → Loads environment variables from .env
#
# If you have the venv active (a2a-examples kernel selected) these are already
# installed. Run this cell anyway to ensure the kernel is up to date.
# ─────────────────────────────────────────────────────────────────────────────
%pip install "a2a-sdk[http-server]" httpx python-dotenv --quiet
print("✅ Dependencies ready")

In [ ]:
import os
from dotenv import find_dotenv, load_dotenv

# ── Load .env ─────────────────────────────────────────────────────────────
# The A2A client (this notebook) does NOT call GitHub Models directly —
# it talks to the A2A server, which holds the GITHUB_TOKEN.
# We still load .env here for consistency and future lessons.
#
# Reminder: The Lesson 06 server must be running on localhost:10001
#   cd _examples/a2a
#   .\scripts\run_server.ps1     # Windows
#   bash scripts/run_server.sh   # Linux / macOS
# ─────────────────────────────────────────────────────────────────────────────
env_path = find_dotenv(raise_error_if_not_found=False)
if env_path:
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")
else:
    print("ℹ  No .env found — that is OK for the client notebook")

print()
print("⚠  This notebook requires the Lesson 06 server running on http://localhost:10001")
print("   Start it: .\\scripts\\run_server.ps1  (or bash scripts/run_server.sh)")

## Step 2 — Discover the Agent

Every A2A agent serves its Agent Card at `/.well-known/agent.json`.
The `A2ACardResolver` fetches and parses this automatically.

This is the **discovery** step — your client learns the agent's
name, skills, supported modes, and capabilities.


In [ ]:
import httpx
from a2a.client import A2ACardResolver

BASE_URL = "http://localhost:10001"

async with httpx.AsyncClient() as httpx_client:
    resolver = A2ACardResolver(
        httpx_client=httpx_client,
        base_url=BASE_URL,
    )
    agent_card = await resolver.get_agent_card()

print(f"Agent Name:    {agent_card.name}")
print(f"Description:   {agent_card.description}")
print(f"Version:       {agent_card.version}")
print(f"Streaming:     {agent_card.capabilities.streaming}")
print(f"Skills:        {[s.name for s in agent_card.skills]}")

## Step 3 — Create the A2A Client

The `A2AClient` wraps httpx and provides typed methods for
sending messages. It needs the Agent Card (from discovery)
and an httpx client.


In [ ]:
from a2a.client import A2AClient

httpx_client = httpx.AsyncClient()

client = A2AClient(
    httpx_client=httpx_client,
    agent_card=agent_card,
)

print(f"✅ A2AClient ready — targeting {agent_card.url}")

## Step 4 — Helper: Build a Message Request

Every A2A message follows JSON-RPC 2.0 format:

```json
{
  "jsonrpc": "2.0",
  "id": "<unique-id>",
  "method": "message/send",
  "params": {
    "message": {
      "role": "user",
      "parts": [{ "kind": "text", "text": "..." }],
      "messageId": "<unique-id>"
    }
  }
}
```

The SDK provides `SendMessageRequest` and `MessageSendParams` to construct this.


In [ ]:
from uuid import uuid4
from a2a.types import MessageSendParams, SendMessageRequest


def build_request(question: str) -> SendMessageRequest:
    """Build a blocking SendMessageRequest for the given question."""
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": question}],
            "messageId": uuid4().hex,
        }
    }
    return SendMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


print("✅ build_request() helper defined")

## Step 5 — Blocking Call (message/send)

A blocking call waits for the agent to finish processing and
returns the complete response. The response is a `Task` object
with status, messages, and artifacts.


In [ ]:
request = build_request("What is the annual deductible?")

response = await client.send_message(request)

# The response has a .result which is a Task (on success)
if hasattr(response, 'result') and response.result:
    task = response.result
    print(f"Task ID:     {task.id}")
    print(f"Task Status: {task.status.state}")
    print()

    # Extract text from the agent's messages
    if task.status.message:
        for part in task.status.message.parts:
            if part.root.kind == "text":
                print(f"Agent says: {part.root.text}")
else:
    print(f"Error: {response}")

## Step 6 — Parse the Response Helper

Let's create a reusable function to extract text from any A2A response.


In [ ]:
def extract_text(response) -> str:
    """Extract text content from an A2A response."""
    if not hasattr(response, 'result') or not response.result:
        return f"Error: {response}"

    task = response.result
    texts = []

    # Check status message
    if task.status and task.status.message:
        for part in task.status.message.parts:
            if part.root.kind == "text":
                texts.append(part.root.text)

    # Check artifacts
    if task.artifacts:
        for artifact in task.artifacts:
            for part in artifact.parts:
                if part.root.kind == "text":
                    texts.append(part.root.text)

    return "\n".join(texts) if texts else "(no text in response)"


print("✅ extract_text() helper defined")

## Step 7 — Ask Multiple Questions

Test the agent with several questions to verify it's answering
from the knowledge base.


In [ ]:
questions = [
    "What is the monthly premium?",
    "Are cosmetic procedures covered?",
    "How do I file a claim?",
    "What is the meaning of life?",   # not in knowledge base
]

for q in questions:
    request = build_request(q)
    response = await client.send_message(request)
    answer = extract_text(response)
    print(f"Q: {q}")
    print(f"A: {answer}")
    print("-" * 60)

## Step 8 — Streaming Call (message/stream)

Streaming returns partial results as Server-Sent Events (SSE).
This is useful for real-time UI updates.

The `SendStreamingMessageRequest` is identical to `SendMessageRequest`
except the method is `message/stream`.


In [ ]:
from a2a.types import SendStreamingMessageRequest


def build_streaming_request(question: str) -> SendStreamingMessageRequest:
    """Build a streaming request for the given question."""
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": question}],
            "messageId": uuid4().hex,
        }
    }
    return SendStreamingMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


streaming_request = build_streaming_request("Explain the claims process step by step.")

print("Streaming response:")
print("=" * 60)

async for event in client.send_message_streaming(streaming_request):
    # Each event is a Task or TaskStatusUpdate or TaskArtifactUpdate
    print(f"  Event type: {type(event).__name__}")
    if hasattr(event, 'result') and event.result:
        result = event.result
        if hasattr(result, 'status') and result.status and result.status.message:
            for part in result.status.message.parts:
                if part.root.kind == "text":
                    print(f"  Text: {part.root.text[:100]}...")

print("=" * 60)
print("✅ Streaming complete")

## Step 9 — Error Handling

A2A uses JSON-RPC error codes. Common errors:

| Code   | Meaning             |
| ------ | ------------------- |
| -32700 | Parse error         |
| -32600 | Invalid request     |
| -32601 | Method not found    |
| -32001 | Task not found      |
| -32002 | Task not cancelable |

Let's handle errors properly:


In [ ]:
async def safe_query(client: A2AClient, question: str) -> str:
    """Send a question and handle errors gracefully."""
    try:
        request = build_request(question)
        response = await client.send_message(request)

        # Check for JSON-RPC error
        if hasattr(response, 'error') and response.error:
            return f"[Error {response.error.code}] {response.error.message}"

        return extract_text(response)

    except httpx.ConnectError:
        return "[Connection Error] Is the server running on localhost:10001?"
    except Exception as e:
        return f"[Unexpected Error] {type(e).__name__}: {e}"


# Test it
answer = await safe_query(client, "What medications are excluded?")
print(f"Answer: {answer}")

## Step 10 — Experiment!

Try your own questions below. The agent can only answer based
on the ACME Insurance policy document loaded in the server.


In [ ]:
# Change this question and re-run the cell
my_question = "What happens if I miss a premium payment?"

answer = await safe_query(client, my_question)
print(f"Q: {my_question}")
print(f"A: {answer}")

## Cleanup


In [ ]:
await httpx_client.aclose()
print("✅ httpx client closed")

## Summary

| Concept              | What You Learned                                   |
| -------------------- | -------------------------------------------------- |
| **Discovery**        | `A2ACardResolver` fetches the Agent Card           |
| **Blocking calls**   | `send_message()` returns a complete Task           |
| **Streaming**        | `send_message_streaming()` yields partial results  |
| **Response parsing** | Navigate Task → status → message → parts           |
| **Error handling**   | Check `response.error` + catch connection failures |

## Next Steps

You've completed the full A2A loop: **Agent → Server → Client**.

In Lesson 08, you'll build a more complex multi-step agent using
Google's Agent Development Kit (ADK) — and expose it over A2A.

→ Continue to [Lesson 08 — Health Research Agent (ADK)](../08-health-research-adk/)
